In [1]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_anthropic langchain_experimental tavily-python langchain_community


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='prod.env')

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_PROJECT"] = "Code Analysis AI Tool"

In [23]:
from langchain_community.tools.tavily_search import TavilySearchResults
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

from typing import Annotated

from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

import json

from langchain_core.messages import ToolMessage

from typing import Literal


import os
from dotenv import load_dotenv
import streamlit as st
from streamlit_ace import st_ace, LANGUAGES, THEMES, KEYBINDINGS
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.output_parsers import RegexParser
from langchain_community.document_loaders import DirectoryLoader



2024-12-08 01:53:49.668 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [25]:
st.set_page_config(page_title="Code Generation and Analysis AI Tool", layout="wide")
st.title("Code Generation and Analysis AI Tool")
st.write("Write your code in the editor below. Click **Generate Suggestion** to see the AI's suggestion.")

2024-12-08 01:54:56.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:56.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:56.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:56.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:56.782 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:56.784 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:56.785 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
code_analysis_prompt_template = """You are a coding assistant who helps users to analyze and improve their code. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
When you answer with the suggested changes, make sure you also include the relevant code snippets in the relevant language and provide each suggestion and code snippet on a separate line.
The output should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer:"""

output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)

PROMPT = PromptTemplate(
    template=code_analysis_prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser
)

code_analysis_chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True, prompt=PROMPT)

@st.cache_resource
def load_and_split_code(folder_path):
    loader = DirectoryLoader(folder_path, glob="**/*.*")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=100, length_function=len)
    return text_splitter.split_documents(documents)

@st.cache_resource
def create_embeddings(_texts):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(_texts, embeddings)
    return vector_store

code_folder = st.text_input("Enter the path to the folder containing your code files (optional):")
enable_analysis = False
if code_folder:
    if os.path.exists(code_folder) and os.path.isdir(code_folder):
        st.success(f"Folder `{code_folder}` selected.")
        enable_analysis = True
    else:
        st.error("Invalid folder path. Please check and try again.")

if enable_analysis:
    with st.spinner("Loading and splitting code files..."):
        print(code_folder)
        texts = load_and_split_code(code_folder)

    with st.spinner("Creating embeddings..."):
        vector_store = create_embeddings(texts)

if enable_analysis:
    st.subheader("Query Your Codebase")
    query = st.text_input("Enter your question or search term:")
    if query:
        with st.spinner("Searching for relevant code..."):
            relevant_chunks = vector_store.similarity_search_with_score(query, k=2)
            chunk_docs = [chunk[0] for chunk in relevant_chunks]
            results = code_analysis_chain({"input_documents": chunk_docs, "question": query})
            text_reference = "".join([doc.page_content for doc in results["input_documents"]])
            print(f"Answer: {results['output_text']}\n\nReference: {text_reference}")
            st.markdown(f"```Answer: {results["output_text"]}```")


@tool
def code_analysis_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

code_analysis_tool.invoke("print('Hello World')")

2024-12-08 01:54:59.495 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:59.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:59.498 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:59.501 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:59.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 01:54:59.503 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


"Successfully executed:\n```python\nprint('Hello World')\n```\nStdout: Hello World\n\n\nIf you have completed all tasks, respond with FINAL ANSWER."

In [ ]:
tools = [code_analysis_tool]

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

llm = ChatOpenAI(model_name = 'gpt-4o', temperature = 0.3)
# Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

In [ ]:
class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}


tool_node = BasicToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

In [ ]:
def route_tools(
    state: State,
):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END


# The `tools_condition` function returns "tools" if the chatbot asks to use a tool, and "END" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", END: END},
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [19]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)
            
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: The Eiffel Tower is approximately 324 meters (1,063 feet) tall, including its antennas.
Assistant: Certainly! Here's a Python implementation of the merge sort algorithm:

```python
def merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2
        left_half = arr[:mid]
        right_half = arr[mid:]

        # Recursive call on each half
        merge_sort(left_half)
        merge_sort(right_half)

        # Two iterators for traversing the two halves
        i = 0
        j = 0
        
        # Iterator for the main list
        k = 0
        
        # Until we reach either end of either left_half or right_half
        while i < len(left_half) and j < len(right_half):
            if left_half[i] < right_half[j]:
                # The value from the left_half has been used
                arr[k] = left_half[i]
                # Move the iterator forward
                i += 1
            else:
                arr[k] = right_half[j]
                j += 1
      